In [1]:
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import sys
import os
import random
import numpy as np

import warnings
warnings.filterwarnings('ignore')


def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_everything(42)

# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
num_classes = 10

# Convert class labels to one-hot vectors
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Normalize pixel values between 0 and 1
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Define the model architecture
model = keras.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation="relu", input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation="relu"))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation="relu"))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(num_classes, activation="softmax"))

# Compile the model
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

# Choose hyperparameters
batch_size = 64
epochs = 20

# Save the current sys.stdout
original_stdout = sys.stdout

# Open a file for writing
file_stdout = open('training_log.txt', 'w')
# Redirect sys.stdout to the file
sys.stdout = file_stdout

# Start the timer
start_time = time.time()

# Train the model
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

# Calculate the training time
end_time = time.time()
training_time = end_time - start_time

2023-05-22 17:52:14.050316: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-22 17:52:14.222567: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-22 17:52:14.223816: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-22 17:52:15.224754: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-22 17:52:19.566881: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-05-22 17:52:21.352048: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 552960000 exceeds 10% 

KeyboardInterrupt: 

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(x_test, y_test, verbose=2)
test_f1 = 2 * (test_precision * test_recall) / (test_precision + test_recall)

print(f"""
accuracy = {test_accuracy:.2f},
precision = {test_precision:.2f},
recall = {test_recall:.2f},
F1 score = {test_f1:.2f}
""")

# Print the training time
print(f'training time: {training_time:.2f} seconds')

313/313 - 1s - loss: 1.0800 - accuracy: 0.7081 - precision_3: 0.7380 - recall_3: 0.6829 - 1s/epoch - 4ms/step

accuracy = 0.71,
precision = 0.74,
recall = 0.68,
F1 score = 0.71

training time: 425.02 seconds
